# Shahar Michaeli - Homework 1


# Imports

In [1]:
import torch
from torch import nn
import numpy as np

# Required Functions

In [2]:
class BTU(torch.nn.Module):
  def __init__(self, T=0.2):
      super(BTU, self).__init__()
      self.T = T

#                make sure input is tensor / make sure returned value is tensor
#                               V               V
  def forward(self, input: torch.Tensor) -> torch.Tensor:
      return 1 / (1 + torch.exp(-input/self.T))

In [34]:
class Linear2(torch.nn.Module):
  def __init__(self, in_features: int, out_features: int, bias: bool = True, device=None, dtype=None) -> None:
    factory_kwargs = {'device': device, 'dtype': dtype}
    super(Linear2, self).__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.weight = nn.Parameter(torch.empty((in_features, out_features), **factory_kwargs))
    if bias:
        self.bias = nn.Parameter(torch.empty(out_features, **factory_kwargs))
    else:
        self.register_parameter('bias', None)
    self.reset_parameters()

  def reset_parameters(self) -> None:
    self.weight = nn.Parameter(torch.rand([self.in_features, self.out_features]))
    if self.bias is not None:
      self.bias = nn.Parameter(torch.rand([self.out_features]))

  def set_weights(self, w, b):
    weight = torch.tensor(w)

    # print(tuple(weight.shape)==(self.in_features,self.out_features), len(b)==self.out_features)
    if weight.shape != (self.in_features,self.out_features) or len(b) != self.out_features:
      raise Exception('Weight shape not equals to (in_features,out_features) or Bias shape is not equal to out_features shape!')

    self.weight = nn.Parameter(weight)
    self.bias = nn.Parameter(torch.tensor(b))
    
  def forward(self, input: torch.Tensor) -> torch.Tensor:
    return torch.matmul(input, self.weight) + self.bias # * is elementwise

  def extra_repr(self) -> str:
    return 'in_features={}, out_features={}, bias={}'.format(
        self.in_features, self.out_features, self.bias is not None
      )
  

In [4]:
class Network3(nn.Module):
  def __init__(self, k, in_dim, out_dim,bypass=True):
    super().__init__()
    self.bypass = bypass
    self.hidden = Linear2(in_dim, k)
    if self.bypass:
      self.output = Linear2(k + in_dim, out_dim)
    else:
      self.output = Linear2(k, out_dim)
    self.BTU = BTU(Temp)
  
  def set_weights(self, w, b, layer):
    if layer == 'output':
      self.output.set_weights(w, b)
    elif layer == 'hidden':
      self.hidden.set_weights(w,b)

  def forward(self, input):
    z1 = self.hidden(input)
    y1 = self.BTU(z1)

    if self.bypass:
      y1_concat = torch.cat((input, y1), 1)
      z2 = self.output(y1_concat)

    else:
      z2 = self.output(y1)
    return self.BTU(z2)



In [5]:
def Loss(model,x , t, print_deltas=False):
  squared_deltas = torch.square(model(x) - t) ## SSE
  if print_deltas:
    print(squared_deltas)
  return torch.sum(squared_deltas)

# Data

In [9]:
Temp = 0.001  # for sigmoid
x = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
t = torch.tensor([[0.], [1.], [1.], [0.]], dtype=torch.float32)

# K = 4 , n = 2
![image](https://user-images.githubusercontent.com/49268387/137798058-83577ff7-05a4-4eef-81cd-03e70f7abf19.png)



In [36]:
model_k4_n2 = Network3(k = 4,in_dim = 2, out_dim=1, bypass=False)
model_k4_n2.set_weights([[-1.],[1.],[1.],[-1]],[-0.5],'output')
model_k4_n2.set_weights([[-1.,-1.,1.,1.],[-1.,1.,-1.,1.]],[0.5,-0.5,-0.5,-1.5],'hidden')


True True
True True


In [37]:
print(model_k4_n2(x))

tensor([[0.],
        [1.],
        [1.],
        [0.]], grad_fn=<MulBackward0>)


In [38]:
Loss(model_k4_n2,x,t,True)

tensor([[0.],
        [0.],
        [0.],
        [0.]], grad_fn=<PowBackward0>)


tensor(0., grad_fn=<SumBackward0>)

# K = 2 , n = 2
![image](https://user-images.githubusercontent.com/49268387/137798430-8e3d4dcf-029d-4865-b3a4-78b57cf871b8.png)


In [39]:
model_k2_n2 = Network3(k = 2,in_dim = 2, out_dim=1, bypass=False)
model_k2_n2.set_weights([[1.],[1.]],[-0.5],'output')
model_k2_n2.set_weights([[-1.,1.],[1.,-1.]],[-0.5,-0.5],'hidden')

True True
True True


In [40]:
print(model_k2_n2(x))

tensor([[0.],
        [1.],
        [1.],
        [0.]], grad_fn=<MulBackward0>)


In [41]:
Loss(model_k2_n2,x,t,True)

tensor([[0.],
        [0.],
        [0.],
        [0.]], grad_fn=<PowBackward0>)


tensor(0., grad_fn=<SumBackward0>)

# K = 1 , n = 2
![image](https://user-images.githubusercontent.com/49268387/137798854-137ede0b-40c1-42e8-a387-0f0af111673b.png)


In [42]:
model_k1_n2 = Network3(k = 1,in_dim = 2, out_dim=1, bypass=True)
model_k1_n2.set_weights([[1.],[1.]],[-1.5],'hidden')
model_k1_n2.set_weights([[1.],[1.],[-2.]],[-0.5],'output')


True True
True True


In [43]:
print(model_k1_n2(x))

tensor([[0.],
        [1.],
        [1.],
        [0.]], grad_fn=<MulBackward0>)


In [44]:
Loss(model_k1_n2,x,t,True)

tensor([[0.],
        [0.],
        [0.],
        [0.]], grad_fn=<PowBackward0>)


tensor(0., grad_fn=<SumBackward0>)

In [45]:
%%shell
jupyter nbconvert --to html /content/Homework_1_Shahar_Michaeli.ipynb


[NbConvertApp] Converting notebook /content/Homework_1_Shahar_Michaeli.ipynb to html
[NbConvertApp] Writing 308521 bytes to /content/Homework_1_Shahar_Michaeli.html
